# First Neural Network

## Overview

This exercise makes use of a large dataset of IMDB movie reviews that is available as a public dataset.  The activity itself is the attempt to create a deep learning neural network that can successfully classify reviews as either positive or negative.  When you are classifying in this way, you are performing a *binary classification* which is often referred to as a *sentiment classification*.

## Goals

By the end of this lab, you should be able to:

* Understand and use *Multi-Hot Encoding*
* Create a TensorFlow model using dense layers
* Understand what the *Bag of Words* approach is and how it is applied
* See the difference between "Machine Learning" and "Understanding"

## Estimated Time: 45 - 60 minutes

# Importing the Libraries

In this lab, we will use a *toy dataset* that is included as a part of the Keras package.  What is a toy dataset?

Don't let the name fool you.  Toy datasets are useful data, but they are datasets that are both free and that represent currently well understood problems.  These datasets are often used for learning the fundamentals of machine learning.  They are also widely used when developing new approaches and algorithms for machine learning.

Toy datasets are particularly useful when developing new algorithms.  Since these are generally considered to be "solved problems," we have many examples and a great deal of literature that we can use to evaluate and compare our new approach or algorithm.  We will not be using the dataset in this way; our goal is to build our first neural network without worrying too much about preprocessing the data.

## What We Need

Our solution will make use of:

| Library/Package | Typical Alias | Functions/Classes Required |
|:---------------:|:-------------:|:--------------------------:|
| numpy | np | |
| tensorflow | tf | |
| tensorflow.keras | | models, layers, datasets |
| matplotlib.pyplot | plt | |

The Keras package, which previously was separate from TensorFlow, is a convenience library that allows us to build common neural network layers using simple convenience functions.  While we will create more advanced networks later in the course, for now we will stick with the simplest methods for building networks.

# <img src="../images/task.png" width=20 height=20> Task 2.1

Please use the following cell to import all of the requirements listed above.

## The Data

We will be using a dataset from the IMDB movie review database.  The IMDB review data is stored as two tuples of tuples.  These are a first tuple representing a set of 25,000 movie reviews with the associated 25,000 labels.  The second tuple contains a set of another 25,000 movie reviews along with the matching labels.  Here is the documentation provided in the `load_data` function that is provided by `datasets.imdb.load_data()`:

```
load_data(path='imdb.npz', num_words=None, skip_top=0, maxlen=None, seed=113, 
    start_char=1, oov_char=2, index_from=3, **kwargs)
    Loads the IMDB dataset.
    
    Arguments:
        path: where to cache the data (relative to `~/.keras/dataset`).
        num_words: max number of words to include. Words are ranked
            by how often they occur (in the training set) and only
            the most frequent words are kept
        skip_top: skip the top N most frequently occurring words
            (which may not be informative).
        maxlen: sequences longer than this will be filtered out.
        seed: random seed for sample shuffling.
        start_char: The start of a sequence will be marked with this character.
            Set to 1 because 0 is usually the padding character.
        oov_char: words that were cut out because of the `num_words`
            or `skip_top` limit will be replaced with this character.
        index_from: index actual words with this index and higher.
        **kwargs: Used for backwards compatibility.
    
    Returns:
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    
    Raises:
        ValueError: in case `maxlen` is so low
            that no input sequence could be kept.
    
    Note that the 'out of vocabulary' character is only used for
    words that were present in the training set but are not included
    because they're not making the `num_words` cut here.
    Words that were not seen in the training set but are in the test set
    have simply been skipped.
```

The movie reviews were originally sentences made up of words, but these have been preprocessed to save us some time.  The words have been replaced with numbers that represent an index into the set of words used in all reviews.  For example, imagine that we had the following index:

1. The
2. fox
3. brown
4. is
5. quick
6. lazy
7. jumps
8. dog
9. lazy
10. jumped
11. over

Given that index, the following sentence:

```
the quick brown fox jumps over the lazy dog
```

would be represented in our dataset as:

```
1 5 3 2 7 11 1 6 8
```

The table of words is ordered by frequency, meaning that word with index 1 occurs more frequently than any other word in the dataset.  If a word has an index of 0, this simply means that the word was unique in the data and was not stored.  Having the words organized in this way allows us to limit how many "features" (words) our network will have to work with.

***This is not exactly how the IMDB data is stored,*** but it is close.  The IMDB data includes three special values at the beginning of the dictionary in indices 0, 1, and 2.

Let's start by loading the data.

# <img src="../images/task.png" width=20 height=20> Task 2.2

Use the imported `datasets` package to load the IMDB data.  The function to load the data can be accessed using `datasets.imdb.load_data()`.  Remember that this function will return a tuple of tuples.  Please assign the first tuple to variables named `train_data` and `train_labels`.  Assign the second tuple to variables named `test_data` and `test_labels`.  When importing the data, specify the `num_words` value as 10000.

When you load this data, you receive some *deprecation warnings*.  Deprecation warnings inform you about features that are no longer supported (deprecated) and may be removed in the future.  If this were our own dataset we would be concerned about addressing these warnings so that our data will remain accessible in the future.  Since we are working with a toy dataset that is included in the library, we're not so concerned.

The warning that we receive is being generated because the data being returned is comprised of tuples where the arrays within the tuples are different shapes.  You can still create *ragged arrays* or *ragged sequences* (arrays or sequences with different shapes that are tightly associated) but the correct way to do so today is to define the type of the arrays as `object` rather than integers or floating point data, which these arrays contain.

# <img src="../images/task.png" width=20 height=20> Task 2.3

Let's examine the data as it is now.  Use the following cell to write code to:

 * Print the shape of the `train_data`
 * Print the shape of the `train_labels`
 * Print the first row of the `train_data`
 * Print the first element of the `train_labels`

Currently, the data is very difficult to read.  Let's load in the dictionary of words that goes with the reviews so that we can make sense of the reviews.  The word index is provided via the `get_word_index()` method on the `imdb` dataset.

# <img src="../images/task.png" width=20 height=20> Task 2.4

Use the `get_word_index()` method offered by the `imdb` dataset in Keras.  Assign the index to a variable named `word_index`.  What datatype is the index?  What does it contain?

The dictionary contains all of the words found in any of the reviews in the dataset.  Each word has been used as a key in the dictionary.  The values in the dictionary represent the position of the word when they are considered in order of frequency.

# <img src="../images/task.png" width=20 height=20> Task 2.5

Currently, we have a word index that allows us to look up the word number when we know the word.  What we have are reviews that are made up of word numbers.  To make sense of the reviews we need a dictionary that uses the word number as the key and the word as the value.

Write Python code to create a dictionary named `reverse_word_index` that inverts the `word_index` dictionary that we already have.  The keys should be the word numbers and the values should be the words.


It would also be very useful to have a convenience function that can convert a review made up of word indices into a review made up of words.

# <img src="../images/task.png" width=20 height=20> Task 2.6

Write a function that will take a review as represented in the dataset and return the review as words.  When you have completed the function, print the entire first review in the `train_data` array.

***NOTE:*** It is nearly 100% likely that your decoded review *will not make sense* on your first try.  The reason is that the values 0, 1, and 2 in the reviews are used to indicate some special conditions:

 * 0 = Padding
 * 1 = Start of review
 * 2 = Word not loaded

This means our code must account for these three extra values when decoding.  There are many ways to solve this problem.  If you get stuck, ask for help or take a peek at the sample solution code.

We now have readable reviews.  We can see that part of the preprocessing was the removal of commas and periods, though contractions were preserved.  Let's create one more convenience function.

# <img src="../images/task.png" width=20 height=20> Task 2.7

Create a Python convenience function that returns the word "Positive" or "Negative" based on whether the value passed in is a 1 or a 0.  We will use this to turn the training label into something readable.

It is almost time to build our network and start training.  There remains a problem to overcome first.

# <img src="../images/task.png" width=20 height=20> Task 2.8

Print out the lengths of the first 15 reviews.

## Vectorization
As you can see, the reviews are all different lengths!  You may recall from our class discussion that the main approaches to supervised machine learning make heavy use of Linear Algebra.  When representing our data for use in these models, we need to represent them as *vectors*, and those vectors must all be the same shape.

To solve this problem, we must *vectorize* our reviews.  There are many ways that we might choose to approach this problem.  For example:

 * Choose some review length, perhaps based on the median or the average length, and either truncate or pad all reviews to be of this same length.
 * Expanding on the above, remove all words that do not contribute to the overall sentiment of the review.  For example, words such as 'a', 'the', 'an', etc.  This still follows the pattern in the approach above but ensures we are retaining the most useful words.
 * Use an approach more similar to the Bayesian approach where we iterate over all of the words in the review, updating the sentiment as we proceed through each word (This type of approach is actually *very similar* to how a recurrent neural network works in that we are proceeding through any arbitrary length sequence).
 * Multi-hot encode the data as a static length vector
 
For our first network, we will take the last approach; i.e., multi-hot encode the reviews.  What might this look like?  Consider the following sample review:

> To be or not to be

Imagine that our dictionary is rather small, with only ten words in it.  If we were to multi-hot encode our review, it might be encoded as follows:

| the      | question | is        | a | to | not | and | or | as | be |
|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|
| 0 | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 0 | 1 |

This means that our six word review becomes a ten dimensional vector:

$$\left[ 0, 0, 0, 0, 1, 1, 0, 1, 0, 1 \right]$$

What if our review was 50 words long?  It would still result in a 10 dimensional vector!  This approach seems useful since it will give  uniform vectors, which is what we need.  There are some downsides to this approach:

 * We lose the order of the words in the review
 * We lose detail; if a word repeats, it is only encoded one time
 
The approach that we are taking is known as *Bag of Words.*  Is this a valid approach?  As we know, the answer to whether an approach is valid is less relevant than whether it *works.*. Since there is no good way to rigorously prove that an approach is correct, we base our answer on experimental evidence.  What does this mean?  Let's try it and see how it works!

# <img src="../images/task.png" width=20 height=20> Task 2.9

Write a Python function that takes an arbitrary length array of values and a dimensionality value as input, returning a multi-hot encoded array whose length matches the dimensionality value.

*Hint:* Numpy provides the `zeros()` function, which will generate a numpy array with the specified length that is populated with zeros.

# <img src="../images/task.png" width=20 height=20> Task 2.10

Our code so far will vectorize a single row of data.  Rewrite your function so that you can pass it a numpy array where each row represents the data to be encoded.  Your function should return a vectorized representation of the original array.

*Hint:* The `zeros()` function can take a tuple as input.  The tuple defines the shape of the resulting vector, matrix, or tensor of zeros.

## Prepare the Data
Now that we have our function defined, we're ready to convert the array of reviews into a massive 10,000 dimensional tensor!

# <img src="../images/task.png" width=20 height=20> Task 2.11

Use the vectorization function that you created above to convert the `train_data` and `test_data` into `x_train` and `x_test` vectorized arrays respectively.

### A Word on Dimensions
You might think that more features is better, and in many ways it can be.  However, there's a trade-off.  More features means more dimensions, more dimensions means more memory, more dimensions means more processing time.  Do you remember the ideas of bias and variance?  By using lots and lots of features, we are creating a much more complex model.  By making it overly complex, we are introducing a great deal of variance.  This can lead to false correlations and overtraining or overfitting.

There are other side effects.  Originally, we were going to ask you to adjust the data to use the 20,000 most common words from reviews, but it turns out that the 4 gigabytes of memory allocated to the virtual machine isn't enough to handle something that large!  Even so, it turns out that our accuracy comes in almost exactly the same.  There's a lesson in this; sometimes we can get "Feature Crazed."  We need to stand back and test things from time to time to see if adding features is actually adding value.

### Some Additional Housekeeping
Before we create and train our network, we need to do just a little bit more massaging.  Specifically, we want to convert the training labels and test labels (how the data has been categorized) into Numpy arrays.  When we do this, we will also force the data to be interpreted as floating point values.  This isn't *strictly* necessary, but it's not a bad habit since we are going to be dealing with percentages.

# <img src="../images/task.png" width=20 height=20> Task 2.12

Convert the data in `train_labels` and `test_labels` into numpy arrays that are composed of `np.float32` values.  Assign them to the variables `y_train` and `y_test` respectively.  Verify the conversion.

## Creating and Training a Neural Network

We are *finally* ready to create the neural network!  While the mathematics behind this involves linear algebra and calculus, actually creating the network turns out to be deceptively simple.  We will arbitrarily build a neural network that takes our input, has several *hidden layers*, and an output layer.  Generally, the input and output layers aren't counted as hidden layers since they are exposed.  The layers that have no external interface are called *hidden.*

The layer that accepts our input must be informed about the number of dimensions (features) found in the data that will be passed to it.  We must also select an *activation* function for each of the layers.  This function manipulates the results of our tensor product (yikes) by constraining its final value.  Additionally, while much of the maths involved revolve around Linear Algebra, this function *specifically introduces non-linearity* into the process.  **We'll talk about this more in our next discussion.**

We will use two activation functions:
 * Rectified Linear ("relu") - This function maps the output into a value of zero or more
 * Sigmoid ("sigmoid") - This function maps the output into a value that will never be lower than zero or greater than one.  The higher the value is, the closer to one this function will get, but it will never hit one.  Similarly, the smaller the value, the closer to zero the output, though it will never reach zero.  The name of this function is based on its shape:  Imagine an 'S' where the head and tail have been stretched out to reach from zero to one.

Finally, we will compile our model, applying an optimizer named *rmsprop* and use a loss measurement function known as *binary_crossentropy.*  *Why* isn't so important just now.  We will explore these details in later discussions.

### Making a Model

The type of model that we will make use of is a *sequential* model.  This means that the data propagates through the layers in sequence.  We can create a model of this type by creating an instance of the `models.Sequential()` class.

Once we have created an instance of this type of model, we can use the `add()` method to append layers to the model.  In this first network, we will make use of the `layers.Dense()` layer only.  A *dense* layer is also known as a *fully connected* layer.  This means that every neuron in this layer is connected to every neuron in the preceding and the subsequent layers.

When we create a dense layer, we have to specify the number of neurons that will make up that layer.  There is no real rule that defines how many neurons you should create.  It can seem as though more neurons must be better, but having a very large number of neurons relative to our data can result in memorization of the training data rather than a useful system to translate from the input data to the output data.  Using too few neurons would result in losing important characteristics of the data.

How do you know what values to use?  Again, there are no rules.  Typically, we choose an arbitrary number of neurons based on our intuition and see how it performs.  If it performs well, we can leave things as-is.  If we want to improve performance, we can try adjusting various *hyperparameters.*

What are hyperparameters?  They are effectively everything that isn't within the neurons themselves.  For example, the number of layers, the number of neurons in the layers, the activation functions that we use, the optimizer that we use, etc.

Since designing and training a neural network is ultimately an optimization problem, we are trying to come up with the simplest possible network that gives the greatest possible accuracy with balanced bias and variance.

# <img src="../images/task.png" width=20 height=20> Task 2.13

TensorFlow will randomize all of the starting weights and biases in our network.  Since we want to create several different example models and compare how well they learn, in addition to wanting our output to match the output in the solutions, we will set the random seed used for generating these values.  We will typically do this when we are trying to optimize a model and need to compare how our changes impact how well a network learns.

Run the following cell:

In [ ]:
tf.random.set_seed(42)

# <img src="../images/task.png" width=20 height=20> Task 2.14

Use the following cell to create a sequential model assigned to the variable `model`.  The model should have three layers defined as follows:

 * A `Dense()` layer with 32 neurons using 'relu' as the activation function and defining the `input_shape` keyword parameter to be `(10000,)` which is the shape of our vectors.
 * A `Dense()` layer with 128 neurons using 'relu' as the activation function.
 * A `Dense()` layer with 1 neuron using 'sigmoid' as the activation function.  This will be our output layer.

Assuming that you have successfully written code in the previous cell that runs with no errors, let's have a look at what we've done.

# <img src="../images/task.png" width=20 height=20> Task 2.15

TensorFlow provides a useful function, `model.summary()`  The `summary()` function will print out a summarization of our entire model.  This includes the number of layers, the number of parameters in each layer, the order of the layers, the input to the model, etc.

Use the following cell to print the `summary()` of the model we have created.

The next step is to compile the model and then use the `fit()` method to train the model.

# <img src="../images/task.png" width=20 height=20> Task 2.16

Use the next cell to run the following code:
```
model.compile(optimizer='rmsprop', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])
training_history = model.fit(x_train, 
    y_train, 
    epochs=30, 
    batch_size=2000, 
    validation_split=0.1)
```

***Note:*** We are using some *very* exaggerated values here.  We will try this again with more typical values.

Before we test out our model, let's visualize how the training went.  The `training_history` that is generated by the `fit()` method contains a `history` object that contains dictionary of arrays.  By default this dictionary will contain the `loss` values for each epoch.  Since we specified that we wanted to use `metrics=['accuracy']` when we compiled our model, we will also have an `accuracy` key.  Finally, since we used the `validation_split` option in our `fit()`, we will also have a `val_loss` and `val_accuracy` key.

# <img src="../images/task.png" width=20 height=20> Task 2.17

Use matplotlib to create a utility function to graph the accuracy vs. validation accuracy and the loss vs. the validation loss for any `history` object that you pass to it.  Name your function `plot_history()`.

What exactly do these graphs tell us?  The accuracy values are fairly straightforward.  We can see that the accuracy during training started at around 68% and ended at about 100%.  That sounds great!  Our validation accuracy has not fared so well.  Following the first training epoch, validation accuracy was at around 82%.  It rose nearly to 90% and then trended generally downward to end at around 87%.

The loss, on the other hand, shows a steady downward trend while training, but the validation loss *increases!*. The slight downward trend in validation accuracy coupled with the rising validation loss both tend to indicate that the model is overfitting the data.  Generally, we'd prefer to see the loss go down and stop training before it bounces back up, both for the training loss and validation loss.  In this case, that would mean that we should likely stop training after four epochs with the current model and hyperparameters.

# <img src="../images/task.png" width=20 height=20> Task 2.18

The model that we just created and the values used for fitting were somewhat exaggerated so that we could learn a bit about the training process through the graphs.  Let's try this again.

The next cell redefines the network and retrains it.  Execute the next cell.

In [ ]:
model = models.Sequential()
model.add(layers.Dense(80, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(40, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

training_history = model.fit(x_train, y_train, epochs=15, batch_size=32, validation_split=0.1)
plot_history(training_history.history)

## Using the Neural Network

While the validation accuracy is still under 90%, this model should be more than acceptable.  To make use of the model, we can pass it an array of reviews that we want evaluated.  How do we know this?

Remember that when we defined our model, we specified that the model takes an input vector of shape `(10000,)`.  We must use this same shape for any data that we pass to it.

To make predictions we make use of the `predict()` method on the model.

# <img src="../images/task.png" width=20 height=20> Task 2.19

Use the `predict()` function to generate predictions for the `x_test()` data.  Capture the results in a variable named `predictions`.  What does this variable contain when you are done?

The resulting array in `predictions` has the shape `(25000, 1)`.  Each row is a one dimensional vector (effectively, a scalar).  Each value represents how positive or negative the prediction is for the respective review.

While we have really performed a regression, we can view this, more or less, as a confidence value.  The closer to zero, the more confident the model is that this is a negative review.  The closer to 1, the more positive.

# <img src="../images/task.png" width=20 height=20> Task 2.20

Print the decoded text for the first five test reviews.  Also print the respective label from `y_test` and the prediction.

Looking at each review manually to see how well the model performs would take a lot of time.  We could certainly write some code to do this evaluation, but this is such a common task that it seems that there should be something built into TensorFlow for this purpose.  Indeed, there is!

The `Model` class implements an `evaluate()` method.  While there are a number of possible options, we are primarily interested in passing in *X* and *Y* values and obtaining an overall loss and accuracy.  For this reason, we are also interested in the `verbose` keyword argument.  If we do not set this to zero, the default behavior will be for the `evaluate()` function to print out status data as it processes.

# <img src="../images/task.png" width=20 height=20> Task 2.21

Use the `model.evaluate()` function to measure the accuracy of the model against the entire set of test data.  You can pass in `x` and `y` as keyword arguments, but they are also the first two positional arguments.  Finally, we recommend that you pass `verbose=0` as a keyword argument.

What is the overall accuracy of the model against the test data set?

### An Important Lesson
Read over the first three reviews that were printed above.  Remember, your network is now evaluating reviews that it has never seen before; these are completely different from anything found in the training data.  Looks good!

But wait... read review #4. Our network is fairly sure that this is a positive review, but when you read it, does it sound positive?  *What's going on???*

Clearly, our model is useful, but there is no *understanding*.  Rather, this is really a
statistical method for identifying which word combinations are most likely to result in a negative or positive
review.

**Think about this in terms of applying Machine Learning to information security problems!!!** There is definitely great value in applying these techniques to identify different types of behavior, *but in the end, the neural network really doesn't **understand** what it's looking at!* Additionally, figuring out how to represent the data in a useful way can be challenging!

# Conclusion

In this lab, we built our first deep learning neural network.  This network wasn't very deep, but even so, it was able to perform pretty well on a task that would normally require a human!  The `evaluate()` function tells us that our test data performs just slightly worse than our validation accuracy predicted that it would.  At a higher level, we also learned:

 * What the Bag of Words approach is and how to apply it
 * How to vectorize data through multi-hot encoding
 * How to build a sequential model
 * How to train a TensorFlow model
 * How to generate predictions
 * How to evaluate a model during training and after
 
# Please Continue!!
Please continue straight on to the *4.03 - BOW Ham vs Spam* lab!